In [78]:
import pdfplumber
import re
from PyPDF2 import PdfReader

In [79]:
def remove_first_line(text):
    return text[text.find('\n') + 1:]

In [80]:
def extract_text_pdfplumber(pdf_path):
    full_text = []

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            
            if not text:
                continue

            text = remove_first_line(text) # верхний колонтитул

            # text = re.sub(r"\bГЛАВА\s?\d+\b", "", text, flags=re.IGNORECASE)

            text = re.sub(r"Рис.\s?\d+.\d+.", "", text, flags=re.IGNORECASE) # подписи к рисункам
            
            # text = re.sub(r"^\d+$", "", text, flags=re.MULTILINE)

            full_text.append(text)
    
    return "\n\n".join(full_text)

In [81]:
def extract_text_pypdf2(pdf_path):
    full_text = []

    reader = PdfReader(pdf_path)

    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        
        text = page.extract_text()

        if not text:
            continue

        text = remove_first_line(text)

        text = re.sub(r"Рис.\s?\d+.\d+.", "", text, flags=re.IGNORECASE)  
        
        full_text.append(text)

   
    return "\n\n".join(full_text)

In [82]:
pdf_path = "LakeIV_B-100-105.pdf"

In [83]:
extracted_text_pdfplumber = extract_text_pdfplumber(pdf_path)

extracted_text_pypdf2 = extract_text_pypdf2(pdf_path)

print(extracted_text_pypdf2[:5000])



 Извлекаемый проходной пакер (слева) и освобождающий инструмент (справа)
Как правило, постоянные проходные пакеры, устанавливаемые как на кабе-
ле, так и гидравлически, обладают более высокой работоспособностью в разныхдиапазонахтемпературидавлений,чем любойизизвлекаемыхпакеров. Ихнедо-статкомявляетсянеобходимостьфрезерованиядляудаленияпакераизскважины.По большей части, фрезерование не возбраняется, хотя во многих случаях можетоказаться ненужным. Однако извлечение пакера может потребоваться в случае
отказа пакера или если необходим доступ к полному диаметру колонны обсад-
ных труб под пакером при последующих операциях капитального ремонта.

 Уплотнительный узел с фиксирующим переводником
Извлекаемые проходные пакеры обычно дороже по сравнению с постоянны-
ми пакерами из-за сложности конструкции, позволяющей использовать их при
пониженных давлениях и температурах. Однако в большинстве случаев такиепакеры можно легко извлечь из скважины без фрезерования. Как правило, извле-чение осуществ

In [84]:
pdf_name = pdf_path.split(".")[0]

with open(f"outputs/{pdf_name}_pypdf2.txt", "w", encoding="utf-8") as f:
    f.write(extracted_text_pypdf2)

with open(f"outputs/{pdf_name}_pdfplumber.txt", "w", encoding="utf-8") as f:
    f.write(extracted_text_pdfplumber)